# Import Required Packages

In [1]:
# Basic Packages 
from dotenv import load_dotenv
# This package is useful to load the environment variables

from openai import OpenAI 
# Package that provide endpoints to utilize the llms

import json # json is the best way to communicate with llm's
import os
import requests

from pypdf import PdfReader
import gradio as gr # Open source package for building light weight front end application

/Volumes/Transcend/DeepLearning/.venv_agentic_ai/lib/python3.13/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
# Usual start to load the environment variables
load_dotenv(override = True)
openai = OpenAI()

# PushOver (Notification SetUp)

In [3]:
# To get the email notifications we are going to use pushover package
pushover_user = os.getenv("PUSHOVER_USER")
pushover_token = os.getenv("PUSHOVER_TOKEN")
pushover_url = "https://api.pushover.net/1/messages.json"

In [4]:
def push(message):
    print(f"Push: {message}")
    payload = {'user':pushover_user, "token":pushover_token, "message":message}
    requests.post(pushover_url, data = payload)

# Test Sample
push("Hi") # Devices that are registered under pushover will be getting notification

Push: Hi


# ToolCalls 

1. Define the function
2. Provide the tool definition
3. Handle Tool Calls


#### 1. Tool Functions

In [5]:
def record_user_details(email, name="Name not provided", notes="not provided"):
    """ 
        Function Descr: If the user wants to iteractive we ask for their email, name and any notes if he provides
        :param: email -> str: user email address
               name  -> str: user name
               notes -> str: notes provided by the user
    """
    push(f"Recording interest from {name} with email {email} and notes {notes}")
    return {"recorded":"ok"}



def record_unknown_question(question):
    """
        Function Description: If the user asked for a unknown question we record for that information
        :param : question -> str : question asked by the user
    """
    push(f"Recording {question} that I couldn't answer")
    return {"recorded":"ok"}

#### 2. Tool Definition

LLMs can better understand JSON format for tool calling

In [29]:
record_user_details_json = {
    "name": "record_user_details",
    "description": "Use this tool to record that a user is interested in being in touch and provide an email address",
    "parameters": {
        "type": "object",
        "properties":{
            "email": {
                "type": "string",
                "description": "The email address of this user"
            },
            "name": {
                "type": "string",
                "description": "The name of the user"
            },
            "notes":{
                "type": "string",
                "description": "Any additional information about the conversation that's worth recording to give context"
            }
        },
        "required":["email","name"],
        "additionalProperties": False
    }
}

In [30]:
record_unknow_question_json = {
    "name": "record_unknown_question",
    "description": "Use this tool to record any question that couldn't be answered as you didn't know the answer",
    "parameters":{
        "type":"object",
        "properties": {
        "question": {
            "type": "string",
            "description": "The question that couldn't be answered"
        },
    },
    "required": ["question"],
    "additionalProperties": False
    }
}

In [31]:
tools = [{"type": "function", "function":record_user_details_json},
         {"type": "function", "function":record_unknow_question_json}]

####  3. Handle Tool Calls
- Now, depends on the use case your llm should pick the tool based on the context/situation it's dealing




In [32]:
def handle_tool_calls(tool_calls):
    # :param -> tool_calls : Response generated by LLMs
    results = []
    for tool_call in tool_calls:
        tool_name = tool_call.function.name
        arguments = json.loads(tool_call.function.arguments)
        print(f"Tool called: {tool_name}", flush = True)

        # globals() function gets all the functions that are declared in the python environment
        tool = globals().get(tool_name)
        result = tool(**arguments) if tool else {}
        results.append({"role": "tool", "content":json.dumps(result), "tool_call_id": tool_call.id})
    return results

# Linked Profile Context
# Summary

In [33]:
reader = PdfReader("me/linkedin.pdf")
linkedin = ""
for page in reader.pages:
    text = page.extract_text()
    if text:
        linkedin += text

with open("me/summary.txt", "r", encoding="utf-8") as f:
    summary = f.read()

name = "Ed Donner"

In [34]:
system_prompt = f"You are acting as {name}. You are answering questions on {name}'s website, \
particularly questions related to {name}'s career, background, skills and experience. \
Your responsibility is to represent {name} for interactions on the website as faithfully as possible. \
You are given a summary of {name}'s background and LinkedIn profile which you can use to answer questions. \
Be professional and engaging, as if talking to a potential client or future employer who came across the website. \
If you don't know the answer to any question, use your record_unknown_question tool to record the question that you couldn't answer, even if it's about something trivial or unrelated to career. \
If the user is engaging in discussion, try to steer them towards getting in touch via email; ask for their email and record it using your record_user_details tool. "

system_prompt += f"\n\n## Summary:\n{summary}\n\n## LinkedIn Profile:\n{linkedin}\n\n"
system_prompt += f"With this context, please chat with the user, always staying in character as {name}."


# Agentic Workflow LLMs

In [35]:
def chat(message, history):
    messages = [{'role':'system', 'content':system_prompt}] + \
                    history + \
            [{"role":"user", 'content':message}]
    done = False
    while not done:

        # This is the call to the LLM - see that we pass in the tools json

        response = openai.chat.completions.create(model = "gpt-4o-mini", messages=messages, 
                                                  tools = tools)
        print(response)
        finish_reason = response.choices[0].finish_reason

        # If the LLM wants to call a tool, we do that!
        if finish_reason == "tool_calls":
            message = response.choices[0].message
            tool_calls = message.tool_calls
            results = handle_tool_calls(tool_calls)
            messages.append(message)
            messages.extend(results)
        else:
            done = True
        
    return response.choices[0].message.content

In [36]:
gr.ChatInterface(chat, type = "messages").launch()

* Running on local URL:  http://127.0.0.1:7863
* To create a public link, set `share=True` in `launch()`.


ChatCompletion(id='chatcmpl-BY3rwlEuOpSWs1NXxvX7qFFZc7QMK', choices=[Choice(finish_reason='stop', index=0, logprobs=None, message=ChatCompletionMessage(content='Hello! Thank you for visiting my website. How can I assist you today?', refusal=None, role='assistant', annotations=[], audio=None, function_call=None, tool_calls=None))], created=1747457724, model='gpt-4o-mini-2024-07-18', object='chat.completion', service_tier='default', system_fingerprint='fp_96c46af214', usage=CompletionUsage(completion_tokens=17, prompt_tokens=2363, total_tokens=2380, completion_tokens_details=CompletionTokensDetails(accepted_prediction_tokens=0, audio_tokens=0, reasoning_tokens=0, rejected_prediction_tokens=0), prompt_tokens_details=PromptTokensDetails(audio_tokens=0, cached_tokens=2048)))
ChatCompletion(id='chatcmpl-BY3s6m3yx91fJIgaCQXotlW8Gf04t', choices=[Choice(finish_reason='stop', index=0, logprobs=None, message=ChatCompletionMessage(content="That's great to hear! I'd love to connect. Could you pleas